In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
df = pd.read_csv("../data/processed/2023_pace_laps_with_delta_phase.csv")

In [4]:
model = joblib.load(
    "../models/lap_delta_random_forest_prelap_phase.pkl"
)

In [5]:
FEATURES = [
    "TyreLife",
    "TyreLifeSquared",
    "LapInStint",
    "LapNumber",
    "Stint",
    "Compound_encoded",
    "Driver_encoded",
    "Team_encoded",
    "FuelProxy",
    "RaceProgress",
]

In [6]:
race = "Bahrain Grand Prix"
driver = "VER"
pit_lap = 25
pit_loss_seconds = 22.0

In [7]:
race_df = df[
    (df["RaceName"] == race) &
    (df["Driver"] == driver)
].sort_values("LapNumber")

In [8]:
def predict_lap_delta(row):
    X = row[FEATURES].values.reshape(1, -1)
    return model.predict(X)[0]

In [9]:
def simulate_undercut(race_df, pit_lap):
    laps = []

    for _, row in race_df.iterrows():
        if row["LapNumber"] < pit_lap:
            laps.append(predict_lap_delta(row))

        elif row["LapNumber"] == pit_lap:
            # Pit lap cost
            laps.append(predict_lap_delta(row) + pit_loss_seconds)

        else:
            # After pit: reset tyre & stint
            new_row = row.copy()
            new_row["TyreLife"] = row["LapNumber"] - pit_lap + 1
            new_row["TyreLifeSquared"] = new_row["TyreLife"] ** 2
            new_row["LapInStint"] = new_row["TyreLife"]
            laps.append(predict_lap_delta(new_row))

    return np.sum(laps)


In [10]:
def simulate_overcut(race_df, pit_lap, delay=2):
    laps = []

    for _, row in race_df.iterrows():
        if row["LapNumber"] < pit_lap + delay:
            laps.append(predict_lap_delta(row))

        elif row["LapNumber"] == pit_lap + delay:
            laps.append(predict_lap_delta(row) + pit_loss_seconds)

        else:
            new_row = row.copy()
            new_row["TyreLife"] = row["LapNumber"] - (pit_lap + delay) + 1
            new_row["TyreLifeSquared"] = new_row["TyreLife"] ** 2
            new_row["LapInStint"] = new_row["TyreLife"]
            laps.append(predict_lap_delta(new_row))

    return np.sum(laps)


In [11]:
undercut_time = simulate_undercut(race_df, pit_lap)
overcut_time = simulate_overcut(race_df, pit_lap, delay=2)

print(f"Undercut total delta: {undercut_time:.2f} sec")
print(f"Overcut total delta:  {overcut_time:.2f} sec")

if undercut_time < overcut_time:
    print("✅ Undercut is faster")
else:
    print("✅ Overcut is faster")


C:\Users\kulka\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\kulka\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\kulka\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\kulka\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\kulka\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have

Undercut total delta: -32.11 sec
Overcut total delta:  -31.87 sec
✅ Undercut is faster
